<a href="https://colab.research.google.com/github/sanujsriv/_transformer_models_/blob/main/pretraining_from_scratch_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torch transformers

##### DistilBertTokenizer

In [ ]:
from transformers import DistilBertTokenizer
from transformers import DistilBertConfig, DistilBertForMaskedLM
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from transformers import AdamW
from tqdm import tqdm
import os,json
from torch.utils.data import Dataset

In [ ]:
import os,json
dir = '/home/student_no_backup/sakumar/summarization/MeanSum/datasets/yelp_dataset/'
os.chdir(dir)

In [ ]:
text_data = []

print('Reading review.json.....')
with open("review.json",'r') as f:
    content = f.readlines()
    for c in content:
        text_data.append(json.loads(c)['text'])

max_length = 150
read = 100000
data = text_data[:read]

print('Read'+str(read)+' review.json.....DONE')

Reading review.json.....
Read100000 review.json.....DONE


In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
num_epochs = 100
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
config = DistilBertConfig(vocab_size=tokenizer.vocab_size)
model = DistilBertForMaskedLM(config=config)
encodings = tokenizer(data, padding=True, truncation=True, max_length=512, return_tensors="pt")
train_dataset = TextDataset(encodings)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
optimizer = AdamW(model.parameters(), lr=5e-5)

/home/student_no_backup/sakumar/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model.train()

for epoch in range(num_epochs):
    progress_bar = tqdm(train_dataloader, desc="Epoch {:1d}".format(epoch+1))
    for batch in progress_bar:
        inputs = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        progress_bar.set_postfix({'loss': loss.item()})

Epoch 1:   0%|                                                                                           | 0/3125 [00:02<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'backward'

In [ ]:
import torch
from transformers import DistilBertConfig, DistilBertForMaskedLM, Trainer, TrainingArguments
from datasets import load_dataset

# Load and preprocess the dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Define the DistilBERT configuration
config = DistilBertConfig(
    vocab_size=len(tokenizer.vocab),
    max_position_embeddings=512,
    n_heads=6,
    n_layers=6,
    dim=768,
    hidden_dim=3072,
    dropout=0.1,
    attention_dropout=0.1,
    activation="gelu"
)

# Initialize the model
model = DistilBertForMaskedLM(config=config)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./distilbert-wikitext",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

# Save the final model
trainer.save_model("./distilbert-wikitext-final")

ImportError: cannot import name 'load_dataset' from 'datasets' (unknown location)

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


In [ ]:
import datasets

#### BertWordPieceTokenizer

In [ ]:
from tokenizers import BertWordPieceTokenizer
import torch
import torch.optim as optim
from torch.utils.data import Dataset

In [ ]:
text_data = []
print('Reading review.json.....')
with open("review.json",'r') as f:
    content = f.readlines()
    for c in content:
        text_data.append(json.loads(c)['text'])

max_length = 150
read = 100000
data = text_data[:read]

print('Read'+str(read)+' review.json.....DONE')

Reading review.json.....
Read100000 review.json.....DONE


In [ ]:
# paths = [str(x) for x in Path('data/original').glob('**/*.txt')]

# tokenizer = BertWordPieceTokenizer(
#         clean_text=True,
#         handle_chinese_chars=False,
#         strip_accents=False,
#         lowercase=True
# )
# tokenizer.train(files=paths[:10], vocab_size=30_000, min_frequency=2,
#                     limit_alphabet=1000, wordpieces_prefix='##',
#                     special_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# Assuming `encodings` is your tokenized and encoded text data
train_dataset = TextDataset(encodings)

In [ ]:
# tokens = tokenizer('Hello, how are you?')
# print(tokens)
# tokenizer.decode(tokens['input_ids'])

{'input_ids': [101, 7592, 1010, 2129, 2024, 2017, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


'[CLS] hello, how are you? [SEP]'

In [ ]:
# dataset = Dataset(paths = [str(x) for x in Path('data/original').glob('**/*.txt')][50:70], tokenizer=tokenizer)
# loader = torch.utils.data.DataLoader(dataset, batch_size=8)

# test_dataset = Dataset(paths = [str(x) for x in Path('data/original').glob('**/*.txt')][10:12], tokenizer=tokenizer)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4)

In [ ]:
# from transformers import DistilBertForMaskedLM, DistilBertConfig

# config = DistilBertConfig(
#     vocab_size=30000,
#     max_position_embeddings=514
# )

# model = DistilBertForMaskedLM(config)

In [ ]:
learning_rate = 0.001
beta1 = 0.99
beta2 = 0.999
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loader = torch.utils.data.DataLoader(train_dataset, batch_size=8)
optim = optim.Adam(model.parameters(), learning_rate, betas=(beta1, beta2))

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertConfig, DistilBertForMaskedLM, AdamW
from transformers import DataCollatorForLanguageModeling
# from datasets import load_dataset

# Load and preprocess the dataset
# dataset = load_dataset('wikitext', 'wikitext-103-raw-v1', split='train')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

dataset = dataset.map(encode, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:

# Create a PyTorch DataLoader
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
dataloader = DataLoader(dataset, batch_size=8, collate_fn=data_collator)

# Initialize DistilBERT model
config = DistilBertConfig(vocab_size=tokenizer.vocab_size, n_heads=12, n_layers=6, dim=768)
model = DistilBertForMaskedLM(config=config)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}')


In [ ]:
# epochs = 100

# for epoch in range(epochs):
#     loop = tqdm(loader, leave=True)

#     # set model to training mode
#     model.train()
#     losses = []

#     # iterate over dataset
#     for batch in loop:
#         optim.zero_grad()

#         # copy input to device
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         # predict
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

#         # update weights
#         loss = outputs.loss
#         loss.backward()

#         optim.step()

#         # output current loss
#         loop.set_description(f'Epoch {epoch}')
#         loop.set_postfix(loss=loss.item())
#         losses.append(loss.item())

#     print("Mean Training Loss", np.mean(losses))
#     losses = []
#     loop = tqdm(test_loader, leave=True)

#     # set model to evaluation mode
#     model.eval()

#     # iterate over dataset
#     for batch in loop:
#         # copy input to device
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         # predict
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

#         # update weights
#         loss = outputs.loss

#         # output current loss
#         loop.set_description(f'Epoch {epoch}')
#         loop.set_postfix(loss=loss.item())
#         losses.append(loss.item())
#     print("Mean Test Loss", np.mean(losses))

In [ ]:
# import torch
# from torch.utils.data import DataLoader, Dataset
# from transformers import DistilBertConfig, DistilBertForMaskedLM, AdamW
# from transformers import DataCollatorForLanguageModeling
# # from datasets import load_dataset

# # Load and preprocess the dataset
# # dataset = load_dataset('wikitext', 'wikitext-103-raw-v1', split='train')

# dataset = train_dataset
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# def encode(examples):
#     return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# dataset = dataset.map(encode, batched=True)
# dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# # Create a PyTorch DataLoader
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
# dataloader = DataLoader(dataset, batch_size=8, collate_fn=data_collator)

# # Initialize DistilBERT model
# config = DistilBertConfig(vocab_size=tokenizer.vocab_size, n_heads=12, n_layers=6, dim=768)
# model = DistilBertForMaskedLM(config=config)

# # Move model to GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Set up the optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # Training loop
# num_epochs = 3
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0
#     for batch in dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}')

### DistilBERT pretraining

In [ ]:
from transformers import DistilBertTokenizer
from transformers import DistilBertConfig, DistilBertForMaskedLM
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from transformers import AdamW
from tqdm import tqdm
import os,json
from torch.utils.data import Dataset

2024-02-28 14:37:01.248437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 14:37:02.621314: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
dir = '/home/student_no_backup/sakumar/summarization/MeanSum/datasets/yelp_dataset/'
os.chdir(dir)

In [ ]:
text_data = []
print('Reading review.json.....')
with open("review.json",'r') as f:
    content = f.readlines()
    for c in content:
        text_data.append(json.loads(c)['text'])

max_length = 150
read = 100000
data = text_data[:read]

print('Read'+str(read)+' review.json.....DONE')

Reading review.json.....
Read100000 review.json.....DONE
